In [1]:
%pip install geopy folium certifi

Defaulting to user installation because normal site-packages is not writeable
  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached folium-0.16.0-py2.py3-none-any.whl.metadata (3.6 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached branca-0.7.2-py3-none-any.whl.metadata (1.5 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached xyzservices-2024.4.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached MarkupSafe-2.1.5-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached folium-0.16.0-py2.py3-none-any.whl (100 kB)
   ---------------------------------------- 0.0/164.4 kB ? eta -:-

In [45]:
from geopy.geocoders import Nominatim
import ssl
import certifi
import re

# Function to parse location string
def parse_location(location_string):
    match = re.match(r'\[(\w+)\]\s*(.*)', location_string)
    if match:
        country_code = match.group(1)
        city_name = match.group(2)
        return country_code, city_name
    return None, None

# Function to get coordinates using geopy with custom CA bundle
def get_coordinates(country_code, city_name):
    # Setup SSL context with custom CA bundle
    ctx = ssl.create_default_context(cafile=certifi.where())
    geolocator = Nominatim(user_agent="locode_mapper", ssl_context=ctx)
    
    try:
        location = geolocator.geocode(f"{city_name}, {country_code}")
        if location:
            return location.latitude, location.longitude, location.address
        else:
            return None, None, None
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None

# Location to search
location_name = "[nl] Rotterdam"

# Parse the location
country_code, city_name = parse_location(location_name)

if country_code and city_name:
    # Get coordinates
    lat, lon, address = get_coordinates(country_code, city_name)

    if lat is not None and lon is not None:
        print(f"Coordinates for {location_name}:")
        print(f"Latitude: {lat}")
        print(f"Longitude: {lon}")
        print(f"Address: {address}")
    else:
        print(f"Failed to find coordinates for {location_name}")
else:
    print(f"Invalid location format: {location_name}")


Coordinates for [nl] Rotterdam:
Latitude: 51.9244424
Longitude: 4.47775
Address: Rotterdam, Zuid-Holland, Nederland


In [52]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import ssl
import certifi
import re

# Function to parse location string
def parse_location(location_string):
    match = re.match(r'\[(\w+)\]\s*(.*)', location_string)
    if match:
        country_code = match.group(1)
        city_name = match.group(2)
        return country_code, city_name
    return None, None

# Function to get coordinates using geopy with custom CA bundle
def get_coordinates(country_code, city_name):
    # Setup SSL context with custom CA bundle
    ctx = ssl.create_default_context(cafile=certifi.where())
    geolocator = Nominatim(user_agent="locode_mapper", ssl_context=ctx)
    
    try:
        location = geolocator.geocode(f"{city_name}, {country_code}")
        if location:
            return location.latitude, location.longitude, location.address
        else:
            return None, None, None
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None

# Load the Excel data
file_path = '3b072604-2564-499d-bbdf-d9e101eec92d.xlsx'  # Replace with the correct file path
df = pd.read_excel(file_path)

# List to store the coordinates of each leg
locations = []

for _, row in df.iterrows():
    origin = row['Leg Origin Name']
    destination = row['Leg Destination Name']
    transport_mode = row['Leg Transport Mode']
    
    origin_country, origin_city = parse_location(origin)
    destination_country, destination_city = parse_location(destination)
    
    if origin_country and origin_city and destination_country and destination_city:
        origin_lat, origin_lon, _ = get_coordinates(origin_country, origin_city)
        destination_lat, destination_lon, _ = get_coordinates(destination_country, destination_city)
        
        if origin_lat and origin_lon and destination_lat and destination_lon:
            locations.append(((origin_lat, origin_lon), (destination_lat, destination_lon), transport_mode))
        else:
            print(f"Failed to get coordinates for origin {origin} or destination {destination}")
    else:
        print(f"Invalid format for origin {origin} or destination {destination}")

# Draw the map with the locations
def draw_map(locations):
    if not locations:
        print("No valid locations to plot on the map.")
        return
    
    lat_center = sum([loc[0][0] for loc in locations] + [loc[1][0] for loc in locations]) / (2 * len(locations))
    lon_center = sum([loc[0][1] for loc in locations] + [loc[1][1] for loc in locations]) / (2 * len(locations))
    my_map = folium.Map(location=[lat_center, lon_center], zoom_start=2)
    
    for origin, destination, transport_mode in locations:
        folium.CircleMarker(location=[origin[0], origin[1]], radius=5, color="blue", fill=True, fill_color="blue").add_to(my_map)
        folium.CircleMarker(location=[destination[0], destination[1]], radius=5, color="green", fill=True, fill_color="green").add_to(my_map)
        
        line_color = {
            "ROA": "yellow",
            "SEA": "blue",
            "AIR": "pink",
            "RAI": "gray"
        }.get(transport_mode, "black")
        
        folium.PolyLine(locations=[origin, destination], color=line_color, weight=2.5, opacity=1).add_to(my_map)
    
    my_map.save("shipment_route_map.html")
    return my_map

# Draw the map with the valid locations
shipment_map = draw_map(locations)


Error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Lund%2C+se&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002780AFDD670>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))
Failed to get coordinates for origin [se] Lund or destination [se] Malmö Sturup
Failed to get coordinates for origin [th] Bangkok International or destination [au] Sydney Kingsford Smith Internationa
Failed to get coordinates for origin [de] Frankfurt International or destination [de] München / Munich - Franz Josef Stra
Failed to get coordinates for origin [de] München / Munich - Franz Josef Stra or destination [us] Chicago O'Hare International
Failed to get coordinates for origin [dk] Copenhagen Kastrup or destination [ca] Montreal International Dorval
Failed to get coordinates for origin [ca] Montreal International Dorval or destination [mx] Mexico City Juarez Internat